# Démonstration d'extraction, structuration, chargement et analyse de documents réglementaires issus de fichiers PDF

Nous utilisons la librarie `unstructured` pour le chargement et le partitionnement des documents PDF. Cette librarie permet de nombreuses autres opérations telles que le nettoyage, le découpage, le pré-traitement et l'encodage de documents. Elle fonctionne avec de nombreux formats de fichiers, intègre de nombreuses librairies utilisées pour l'extraction de documents et l'OCR, et s'intègre bien avec les libraries majeures du NLP.

Nous utilisons `langchain` pour le NLP et certaines fonctions de traitement du documents. Nous utilisons la base de données vectorielle `ChromaDB` pour le stockage des documents.

In [54]:
import os

from pathlib import Path

from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.pdf import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma

from unstructured.chunking.title import chunk_by_title
from unstructured.documents.elements import NarrativeText, Title
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import convert_to_dataframe

# Lecture et partitionnement d'un document PDF avec `unstructured`

Lecture et partitionnement d'un des fichiers de règlementation :

In [80]:
os.chdir("/home/notebook-user/")
data_path = Path("data")
all_data_files = [path for path in data_path.glob("*.pdf")]
elements = partition_pdf(filename=all_data_files[0])

df_el = convert_to_dataframe(elements)
df_el

,type,element_id,text,coordinates_points,coordinates_system,coordinates_layout_width,coordinates_layout_height,file_directory,filename,languages,last_modified,page_number,filetype,parent_id
0,Header,2e651e3ecf841e3b1ae4881cf6844de8,16.12.2022,"((42.52082690000003, 51.65037499999994), (42.5...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,1,application/pdf,NaN
1,Header,69374b09b1681162ba2d6bba92d6a167,EN,"((119.509, 51.03004999999996), (119.509, 60.09...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,1,application/pdf,NaN
2,Header,82a8d26c971938ba239ca87e2ce1b2d0,Official Journal of the European Union,"((222.803, 51.65037499999994), (222.803, 61.24...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,1,application/pdf,NaN
3,Title,95db08cf4294a5415cf4e088586aa036,DIRECTIVES,"((255.061, 89.36689999999999), (255.061, 106.4...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,1,application/pdf,82a8d26c971938ba239ca87e2ce1b2d0
4,Title,361c1b105637cbb6d2e115e0c4af2984,DIRECTIVE (EU) 2022/2464 OF THE EUROPEAN PARLI...,"((115.37, 145.364375), (115.37, 154.959875), (...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,1,application/pdf,82a8d26c971938ba239ca87e2ce1b2d0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,NarrativeText,2992d8df910d77f3ebebe8ec5ec13805,Article 4 shall be binding in its entirety and...,"((119.05662375000003, 174.1630268), (119.05662...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,66,application/pdf,3315041decb295c3bb2efe759076dffe
1346,Title,538d72b525c489bdeb5b2845a6a405b2,"Done at Strasbourg, 14 December 2022.","((119.05662375000003, 204.03957559999992), (11...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,66,application/pdf,82a8d26c971938ba239ca87e2ce1b2d0
1347,Title,6c5088e01caf6b4dbd43cb5eabd27e22,For the European Parliament The President R. M...,"((143.9444721, 240.20981284999993), (143.94447...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,66,application/pdf,82a8d26c971938ba239ca87e2ce1b2d0
1348,Title,fcca930b512f46d7ea8c7f2ba5246554,For the Council The President M. BEK,"((376.49621235, 240.20981284999993), (376.4962...",PixelSpace,595.276,841.89,data,CELEX_32022L2464_EN_TXT.pdf,[eng],2024-03-04T10:50:22,66,application/pdf,82a8d26c971938ba239ca87e2ce1b2d0


In [81]:
df_el.type.value_counts()

NarrativeText        636
Header               264
UncategorizedText    220
Title                142
ListItem              69
Footer                19
Name: type, dtype: int64

Nous observons comment `unstructured` sépare les différents éléments du documents en catégories, ici stockés de façon pratique dans un dataframe. De plus, la hiérarchie entre les différents éléments sélectionnés est exprimée à travers la colonne `parent_id`.

Nous pouvons également re-découper un document en blocs afin de pouvoir plus aisément analyser ses différentes parties et faire par exemple des recherches de similarités avec diverses requêtes d'information.

In [5]:
chunks = chunk_by_title(elements)
df_chunks = convert_to_dataframe(chunks)
df_chunks

,type,element_id,text,filename,filetype,languages,last_modified,page_number
0,CompositeElement,336b434ab2cc1a3ea75bd82d91a5c745,16.12.2022\n\nEN\n\nOfficial Journal of the Eu...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,1
1,CompositeElement,61efa1b4c38fa382d1cb5586832fa34c,(Text with EEA relevance)\n\nTHE EUROPEAN PARL...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,1
2,CompositeElement,3d760278127b8712ba08c8615556d06a,Whereas:\n\n(1),CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,1
3,CompositeElement,f3f5fd0fef5416113fd9cd31cda65042,In its communication of 11 December 2019 entit...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,1
4,CompositeElement,8d704dddc25aeae00783481e731f0ebc,conserve and enhance the Union's natural capit...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,1
...,...,...,...,...,...,...,...,...
762,CompositeElement,bf956c21ba0a0ce2af1652a7c466ad10,"2. By 31 December 2028, the Commission shall r...",CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,65
763,CompositeElement,b210c863f15bde1ba92fcbe797c06b7c,"By 31 December 2028, the Commission shall asse...",CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,65
764,CompositeElement,dcec36e15b249023e0527325a46eee63,The report shall be transmitted to the Europea...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,65
765,CompositeElement,f419eabb8f312dbe870d22a87c718ddf,Entry into force and application\n\nThis Direc...,CELEX_32022L2464_EN_TXT.pdf,application/pdf,[eng],2024-03-03T16:04:55,65


# Stockage de documents dans une base de données vectorielle et démonstration de requête de document

Nous chargeons les documents à l'aide de `langchain` :

In [21]:
docs = [UnstructuredPDFLoader(str(file)).load()[0] for file in all_data_files]

Nous initialisons une base de données vectorielle ChromaDB dans laquelle nous chargeons nos documents. Ces derniers sont encodés à l'aide d'un embedding OpenAI : 

In [ ]:
with open("openai_api_key.txt", "r") as f:
    api_token = f.readline().replace("\n", "")
    
embedding = OpenAIEmbeddings(openai_api_key=api_token)
vectorstore = Chroma.from_documents(docs, embedding)

Nous utilisons une similarity search pour quérir le document contenant des directives :

In [83]:
query_docs = vectorstore.similarity_search("DIRECTIVES", k=1)
query_docs[0].metadata["source"]

'data/CELEX_32022L2464_EN_TXT.pdf'

# Résumé d'un extrait de texte issu d'un document

Nous faisons appel à un modèle ChatGPT pour résumer le contenu d'un tronçon de texte (le texte entier est trop grand pour être accepté par l'api) :

In [84]:
llm = ChatOpenAI(temperature=0, api_key=api_token)
chain = load_summarize_chain(llm, chain_type="map_reduce", token_max=30000)

text_splitter = RecursiveCharacterTextSplitter()
split_doc = text_splitter.split_documents([docs[0]])

doc_sample = split_doc[0]
sample_summary = chain.invoke([doc_sample])

print(sample_summary["output_text"])

Directive (EU) 2022/2464 amends regulations related to corporate sustainability reporting in the EU to align with the European Green Deal's goals of achieving a resource-efficient, competitive economy with no net greenhouse gas emissions by 2050. It also addresses the socio-economic impact of the COVID-19 pandemic and aims for a sustainable, inclusive, and fair recovery. The European Commission is committed to climate neutrality by 2050 and restoring biodiversity by 2030.


# Commentaires sur le design d'un DB intelligente pour l'exploration de documents légaux

Un outil permettant le stockage et l'accès à des textes réglementaires devrait être composé d'une base de donnée adaptée, de type vectorielle afin de pouvoir réaliser des recherches de similarité et ainsi récupérer facilement l'information pertinente à l'intérieur du document.

Dans le cadre de textes réglementaires, il serait intéressant de pouvoir séparer ces textes en articles et directives afin de pouvoir facilement se référer à l'un d'eux dans une recherche. Il serait également intéressant de pouvoir facilement naviguer entre ces articles, car un article contient souvent des références vers d'autres.